In [2]:
cd "/suphys/aloe8475"


/suphys/aloe8475


In [3]:
cd "Documents/edamame"

/import/silo2/aloe8475/Documents/edamame


In [4]:
from scipy.io import loadmat, savemat
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import datetime
import networkx as nx
from edamame import *
from tqdm import tqdm_notebook
import os
import edamame.core.wires as wires
from random import choice
import warnings
from IPython.core.debugger import set_trace
import nct
import bct

#warnings.filterwarnings('ignore')

In [5]:
import pickle 
import _pickle as cPickle
import gzip
def compressed_pickle(obj, filename,protocol=-1):
    with gzip.open(filename, 'wb') as f:
        cPickle.dump(obj, f, protocol)

In [6]:
# this_seed2=700
def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

In [7]:
#Select Largest Components
def select_largest_component_new(wires_dict):
    """
    Find and select largest connected component of the original graph G.
    Throws away unconnected components and updates all the keys in wires_dict 
    """
#     def connected_component_subgraphs(G):
#         for c in nx.connected_components(G):
#             yield G.subgraph(c)
    
    wires_dict['G'] = max(connected_component_subgraphs(wires_dict['G']), key=len)
#     set_trace()
    nw = len(wires_dict['G'].nodes())
    nj = len(wires_dict['G'].edges())   
    
    logging.info("The largest component has %5d nodes and %6d edges", nw, nj)

    # Replace values in the dictionary
    wires_dict['number_of_wires']     = nw
    wires_dict['number_of_junctions'] = nj
    wires_dict['xa'] = wires_dict['xa'][wires_dict['G'].nodes()] 
    wires_dict['ya'] = wires_dict['ya'][wires_dict['G'].nodes()] 
    wires_dict['xb'] = wires_dict['xb'][wires_dict['G'].nodes()] 
    wires_dict['yb'] = wires_dict['yb'][wires_dict['G'].nodes()]
    wires_dict['xc'] = wires_dict['xc'][wires_dict['G'].nodes()] 
    wires_dict['yc'] = wires_dict['yc'][wires_dict['G'].nodes()]
 
    # Keep old edge_list
    old_edge_list = [(ii, kk) for ii, kk in  zip(wires_dict['edge_list'][:, 0], wires_dict['edge_list'][:, 1])]
    # Remove old edge list
    wires_dict = wires.remove_key(wires_dict, 'edge_list') 
    # Save indices of intersections in the old graph
    ind_dict = {key:value for value,key in enumerate(old_edge_list)}
    new_edge_list = sorted([kk if kk[0] < kk[1] else (kk[1], kk[0]) for kk in wires_dict['G'].edges()], key=lambda x: x[0])
    # Find intersection between the two sets
    inter = set(ind_dict).intersection(new_edge_list)
    # Retrieve edge indices/positions from the old list
    edges_idx = [ind_dict[idx] for idx in inter]
       
    # These have length equal to number of junctions -- only get the ones we need
    wires_dict['xi'] = wires_dict['xi'][edges_idx] 
    wires_dict['yi'] = wires_dict['yi'][edges_idx] 
    
    # Get contiguous numbering of nodes
    # Build node mapping 
    node_mapping    = {key:value for value, key in enumerate(sorted(wires_dict['G'].nodes()))}
    # This  step also renames edges list
    wires_dict['G'] =  nx.relabel_nodes(wires_dict['G'] , node_mapping)

    # Swap node vertices if vertex 0 is larger than vertex 1, then sort by first element
    wires_dict['edge_list'] = np.asarray(sorted([kk if kk[0] < kk[1] else (kk[1], kk[0]) for kk in wires_dict['G'].edges()], key=lambda x: x[0]))
    
    # Save adjacency matrix of new graph
    wires_dict = wires.remove_key(wires_dict, 'adj_matrix') 
    wires_dict = wires.generate_adj_matrix(wires_dict)

    wire_distances = wires.cdist(np.array([wires_dict['xc'], wires_dict['yc']]).T, np.array([wires_dict['xc'], wires_dict['yc']]).T, metric='euclidean')    
    wires_dict['wire_distances'] = wire_distances

    return wires_dict 

In [8]:
cd "/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks/"

/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks


In [12]:
data=loadmat('avgDegreeVals.mat')
avgDeg=data['avgdeg'][0]

numNodes=data['numNodes'][0]
# [8.906040268456376,
#  16.166666666666668,
#  44.06666666666667,
#  8.846153846153847,
#  15.588628762541806,
#  4.473118279569892,
#  2.296028880866426,
#  9.245674740484429,
#  4.083044982698962,
#  8.280575539568344]

In [14]:
ResultSqu_Grid=[None]*10
ResultSqu_Random=[None]*10

In [15]:
seed=[]
for i in range(len(avgDeg)):
    seed.append(np.random.randint(10000))

In [16]:
#create watts strogatz graphs:
random=[]
grid=[]
temp=[]
temp2=[]
for i in range(len(avgDeg)):
    k=int(np.round(avgDeg[i]))
    temp2.append(nx.watts_strogatz_graph(numNodes[i],k,1,seed=seed[i]))
    temp.append(nx.watts_strogatz_graph(numNodes[i],k,0,seed=seed[i]))
    grid.append(max(connected_component_subgraphs(temp[i]), key=len))
    random.append(max(connected_component_subgraphs(temp2[i]), key=len))

In [17]:
random[6].number_of_edges()

287

In [18]:
temp_rand=[]
temp=[]
shortestPath_Grid=[]
shortestPath_Random=[]
onAmp_Grid=[]
onAmp_Random=[]

for i in range(len(avgDeg)):
    temp_rand.append(getFarthestPairing(nx.adj_matrix(random[i]).todense()))
    temp.append(getFarthestPairing(nx.adj_matrix(grid[i]).todense()))
    shortestPath_Grid.append(nx.shortest_path_length(grid[i],temp[i][0],temp[i][1]))
    shortestPath_Random.append(nx.shortest_path_length(random[i],temp_rand[i][0],temp_rand[i][1]))
    onAmp_Grid.append(shortestPath_Grid[i]/5)
    onAmp_Random.append(shortestPath_Random[i]/5)

In [ ]:
stimulus=[[],[]]
stimulus_Rand=[[],[]]
results_grid=[]
results_rand=[]

for i in range(len(avgDeg)):
    dt = 1e-2
    f=0.5
    Time=5
    stimulus[0]=(stimulus__(biasType='AC',onAmp=onAmp_Grid[i],TimeVector=np.arange(0,Time,dt),f=f))
    stimulus[1]=(stimulus__(biasType='Drain',TimeVector=np.arange(0,Time,dt))) #we don't want this drain to be active during training

    stimulus_Rand[0]=(stimulus__(biasType='AC',onAmp=onAmp_Random[i],TimeVector=np.arange(0,Time,dt),f=f))
    stimulus_Rand[1]=(stimulus__(biasType='Drain',TimeVector=np.arange(0,Time,dt))) #we don't want this drain to be active during training
    stimulus2 = [item for item in stimulus] #go through each list in the list and find the ith item
    stimulus2_Rand=[item for item in stimulus_Rand]

    results_grid.append(runSim(connectivity__(graph=grid[i]),stimulus=stimulus2, contactMode='farthest', T = Time, dt = 0.001, onAmp = onAmp_Grid[i], biasType='AC',f=f,junctionMode='tunneling'))
    results_rand.append(runSim(connectivity__(graph=random[i]),stimulus=stimulus2_Rand, contactMode='farthest', T = Time, dt = 0.001, onAmp = onAmp_Random[i], biasType='AC',f=f,junctionMode='tunneling'))
    period=1/f

2020-05-18 15:56:30,161:INFO:First current path [0, 3, 7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 51, 55, 59, 63, 67, 71, 75, 79, 83, 87, 91, 95, 99, 103, 107, 111, 115, 119, 123, 127, 131, 135, 139, 143, 147, 150] formed at time = 0.743 s.


2020-05-18 15:56:46,810:INFO:First current path [17, 235, 189, 13, 278, 140] formed at time = 0.791 s.


2020-05-18 15:57:03,759:INFO:First current path [0, 5, 13, 21, 29, 37, 45, 53, 61, 69, 77, 85, 93, 101, 109, 117, 125, 133, 141, 146] formed at time = 0.751 s.


2020-05-18 15:57:16,041:INFO:First current path [141, 131, 102, 247, 19] formed at time = 0.772 s.


2020-05-18 15:57:34,823:INFO:First current path [0, 12, 34, 56, 78, 100, 122, 134] formed at time = 0.753 s.


In [192]:
WS_Grid=[None]*len(avgDeg)
WS_Random=[None]*len(avgDeg)
for i in range(len(avgDeg)):
    WS_Grid[i]={'seed':[],'adj_matrix':[],'G':[],'Accuracy':{'Linear Transformation':[],'Mackey Glass':[],'MNIST':[]},'Graph Theory':{'Small World':[],'Modularity':[],'CCoeff':[],'MZ':[],'PCoeff':[],'PL':[]}}
    WS_Random[i]={'seed':[],'adj_matrix':[],'G':[],'Accuracy':{'Linear Transformation':[],'Mackey Glass':[],'MNIST':[]},'Graph Theory':{'Small World':[],'Modularity':[],'CCoeff':[],'MZ':[],'PCoeff':[],'PL':[]}}

In [194]:
nodesList=[[None]*len(avgDeg) for i in range(2)]
for i in tqdm(range(len(avgDeg))):
    for j in range(2): #grid, random
        if j == 0:
            if len(grid[i]) >= 250:
                nodesList[j][i]=[50,100,150,200,250,len(grid[i])]#range(50, len(ws300[i][0])+1,50)
            elif len(grid[i]) >= 200 and len(grid[i]) < 250:
                nodesList[j][i]=[50,100,150,200,len(grid[i])]#range(50, len(ws300[i][0])+1,50)  
            elif len(grid[i]) >= 150 and len(grid[i]) < 200:
                nodesList[j][i]=[50,100,150,len(grid[i])]#range(50, len(ws300[i][0])+1,50)  
            elif len(grid[i]) >= 100 and len(grid[i]) < 150:
                nodesList[j][i]=[50,100,len(grid[i])]#range(50, len(ws300[i][0])+1,50)
        else:
            if len(random[i]) >= 250:
                nodesList[j][i]=[50,100,150,200,250,len(random[i])]#range(50, len(ws300[i][0])+1,50)
            elif len(random[i]) >= 200 and len(random[i]) < 250:
                nodesList[j][i]=[50,100,150,200,len(random[i])]#range(50, len(ws300[i][0])+1,50)  
            elif len(random[i]) >= 150 and len(random[i]) < 200:
                nodesList[j][i]=[50,100,150,len(random[i])]#range(50, len(ws300[i][0])+1,50)  
            elif len(random[i]) >= 100 and len(random[i]) < 150:
                nodesList[j][i]=[50,100,len(random[i])]#range(50, len(ws300[i][0])+1,50)

In [203]:
for i in tqdm(range(len(avgDeg))):
    nwSqu_Random=[None]*len(nodesList[1][i])
    nwSqu_Grid=[None]*len(nodesList[0][i])
    countk=0
    for k in nodesList[1][i]: #loop through sets of nodes for regression
        print('Running Random Network Regression: ' + str(k) + ' nodes')
        ResultSqu_Random=[]
        ResultSqu_Random = nonLinearTrans(results_rand[i],'Square',k, repeats=50) #simulation, type of signal, number of nodes to sample from, number of linear regression repetitions (take avg)
        nwSqu_Random[countk]=(ResultSqu_Random['accuracy'])
        countk=countk+1
        if k == nodesList[j][i][-1]: #if k is equal to the largest node sampling
            WS_Random[i]['Accuracy']['Linear Transformation']=[]
            WS_Random[i]['Accuracy']['Linear Transformation'].append(nwSqu_Random)
            print(WS_Random[i]['Accuracy']['Linear Transformation'])
    countk=0
    for k in nodesList[0][i]:     
        print('Running Random Network Regression: ' + str(k) + ' nodes')
        ResultSqu_Grid=[]
        ResultSqu_Grid = nonLinearTrans(results_grid[i],'Square',k, repeats=50) #simulation, type of signal, number of nodes to sample from, number of linear regression repetitions (take avg)
        nwSqu_Grid[countk]=(ResultSqu_Grid['accuracy'])
        countk=countk+1
        if k == nodesList[j][i][-1]: #if k is equal to the largest node sampling
            WS_Grid[i]['Accuracy']['Linear Transformation']=[]
            WS_Grid[i]['Accuracy']['Linear Transformation'].append(nwSqu_Grid)
            print(WS_Grid[i]['Accuracy']['Linear Transformation'])

Running Random Network Regression: 50 nodes
Running Random Network Regression: 100 nodes
Running Random Network Regression: 150 nodes
Running Random Network Regression: 200 nodes
Running Random Network Regression: 250 nodes
Running Random Network Regression: 300 nodes
[[0.6438537327018645, 0.7576049715506393, 0.7756619392197739, 0.7842916310828033, 0.7868175751836598, 0.7904795678316529]]
Running Random Network Regression: 50 nodes
Running Random Network Regression: 100 nodes
Running Random Network Regression: 150 nodes
Running Random Network Regression: 200 nodes
Running Random Network Regression: 250 nodes
Running Random Network Regression: 300 nodes
[[0.6353957683162891, 0.6480427010683283, 0.6534908370267987, 0.6565611517238078, 0.6566798889320961, 0.65689463398104]]
Running Random Network Regression: 50 nodes
Running Random Network Regression: 100 nodes
Running Random Network Regression: 150 nodes
Running Random Network Regression: 200 nodes
Running Random Network Regression: 250 

In [207]:
for i in range(len(avgDeg)):
    print(WS_Grid[i]['Accuracy']['Linear Transformation'])

[[0.6353957683162891, 0.6480427010683283, 0.6534908370267987, 0.6565611517238078, 0.6566798889320961, 0.65689463398104]]
[[0.6855775662555819, 0.6951486854525855, 0.6989350841476106, 0.7007352266677117, 0.7015646750578877, 0.7018715068603305]]
[[0.7424014819946504, 0.7682894688467151, 0.7867460972688727, 0.7963311955889445, 0.801512278245945, 0.8034356645409424]]
[[0.6359725822297821, 0.6481989463530387, 0.6531485011038108, 0.6559581778914502, 0.6568946339225821, 0.6568946339706717]]
[[0.6858686790580839, 0.6953069248932158, 0.698871987227876, 0.7008274205167959, 0.7016350967109192, 0.7018715068603228]]
[[0.5948249065129361, 0.6016905187233145, 0.6066722403574084, 0.6080945924468639, 0.6082108237726748, 0.6082177720586364]]
[[0.5670507111660705, 0.5670507111660696, 0.5670507111660698, 0.5670507111660694, 0.5670507111660698, 0.5670507111660696]]
[[0.6358719872178039, 0.648225149237903, 0.6542650295297279, 0.6560446476087354, 0.6568946339918686, 0.6568946339701597]]
[[0.5960363417108276,

In [208]:
for i in range(len(avgDeg)):
    print(WS_Random[i]['Accuracy']['Linear Transformation'])

[[0.6438537327018645, 0.7576049715506393, 0.7756619392197739, 0.7842916310828033, 0.7868175751836598, 0.7904795678316529]]
[[0.6863976746065106, 0.6893533814097605, 0.6897030207111747, 0.6898896335544714, 0.6900298704889215, 0.690161724197324]]
[[0.6597455426464696, 0.7252391293199645, 0.7325277867323406, 0.7345367661681812, 0.7358183161886225, 0.7366012010309602]]
[[0.7180739895802642, 0.7476528925302719, 0.7538545473977092, 0.7593928677581114, 0.7659585811032942, 0.7702191212067145]]
[[0.6797673698985502, 0.7322281211010393, 0.7864179266173356, 0.8090788957659776, 0.8243654429589905, 0.8276271536386949]]
[[0.7170678332323939, 0.7728374185102834, 0.7951036878063479, 0.8049285553508276, 0.8127869522803505, 0.8172641879763997]]
[[0.5670699175505672, 0.5670699175505675, 0.5670699175505673, 0.5670699175505673, 0.5670699175505673, 0.5670699175505673]]
[[0.6447500872375919, 0.7827718165655705, 0.7958220063562819, 0.807508507105792, 0.8172588957387947, 0.8236468015022114]]
[[0.71151173004819

In [210]:
#export adj matrices to calculate small worldness in matlab:
adj_mats={"AdjMat_Grid":[],"AdjMat_Random":[]}
for i in range(len(WS_Grid)):
    adj_mats['AdjMat_Grid'].append(dict(grid=nx.adj_matrix(grid[i]).todense()))
    adj_mats['AdjMat_Random'].append(dict(random=nx.adj_matrix(random[i]).todense()))
    WS_Grid[i]['adj_matrix']=nx.adj_matrix(grid[i]).todense()
    WS_Random[i]['adj_matrix']=nx.adj_matrix(random[i]).todense()
savemat('/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/300_WS.mat',adj_mats)

In [218]:
cd "/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity/"

/import/silo2/aloe8475/Documents/CODE/Data/Functional Connectivity


In [219]:
smallworld_ws={'Grid':[],'Random':[]}
temp=loadmat(r'300_WS_smallworld.mat')
smallworld_ws['Grid']=temp['smallworld_grid']
smallworld_ws['Random']=temp['smallworld_random']
del temp

In [220]:
cd "/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks/"

/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks


In [232]:
# WS:
ci = []
pcoeff= []
mz= []
clustering = []
for i in tqdm(range(len(WS_Grid))):
    ci,q=nct.community_louvain(np.squeeze(np.asarray(WS_Grid[i]['adj_matrix'])))
    pcoeff=bct.participation_coef(np.squeeze(np.asarray(WS_Grid[i]['adj_matrix'])),ci)
    mz=bct.module_degree_zscore(np.squeeze(np.asarray(WS_Grid[i]['adj_matrix'])),ci)
    WS_Grid[i]['G']=grid[i]
    clustering=nx.clustering(WS_Grid[i]['G'])
    WS_Grid[i]['Graph Theory']['PL']=dict(nx.all_pairs_shortest_path_length(WS_Grid[i]['G']))
    WS_Grid[i]['Graph Theory']['Modularity']=ci
    WS_Grid[i]['Graph Theory']['Modularity Score']=q
    WS_Grid[i]['Graph Theory']['PCoeff']=pcoeff
    WS_Grid[i]['Graph Theory']['MZ']=mz
    WS_Grid[i]['Graph Theory']['Small World']=smallworld_ws['Grid'][0][i]
    WS_Grid[i]['Graph Theory']['CCoeff']=clustering
    WS_Grid[i]['Graph Theory']['Degree']=nx.degree(WS_Grid[i]['G'])
    WS_Grid[i]['Graph Theory']['AvgPL']=nx.average_shortest_path_length(WS_Grid[i]['G'])

ci = []
pcoeff= []
mz= []
clustering = []
count1=0
count2=0
count3=0   
for i in tqdm(range(len(WS_Random))):
    ci,q=nct.community_louvain(np.squeeze(np.asarray(WS_Random[i]['adj_matrix'])))
    pcoeff=bct.participation_coef(np.squeeze(np.asarray(WS_Random[i]['adj_matrix'])),ci)
    mz=bct.module_degree_zscore(np.squeeze(np.asarray(WS_Random[i]['adj_matrix'])),ci)
    WS_Random[i]['G']=random[i]
    clustering=nx.clustering(WS_Random[i]['G'])
    WS_Random[i]['Graph Theory']['PL']=dict(nx.all_pairs_shortest_path_length(WS_Random[i]['G']))
    WS_Random[i]['Graph Theory']['Modularity']=ci
    WS_Random[i]['Graph Theory']['Modularity Score']=q
    WS_Random[i]['Graph Theory']['PCoeff']=pcoeff
    WS_Random[i]['Graph Theory']['MZ']=mz
    WS_Random[i]['Graph Theory']['Small World']=smallworld_ws['Random'][0][i]
    WS_Random[i]['Graph Theory']['CCoeff']=clustering
    WS_Random[i]['Graph Theory']['Degree']=nx.degree(WS_Random[i]['G'])
    WS_Random[i]['Graph Theory']['AvgPL']=nx.average_shortest_path_length(WS_Random[i]['G'])    

name='WS_networks_LinearTransformation.pkl'
print('Saving WS')
with open(name, 'wb') as f:
    pickle.dump([WS_Grid,WS_Random], f)   

Saving WS
